### Coding puzzles: Backtracking

[Playstore link for game](https://play.google.com/store/apps/details?id=zl.puzzle.riveriq&hl=en_IN)

Code the solution to these games (using recursion / backtracking)

In [18]:
class Dict:
    def __init__(self, d):
        self.dict = d

    # How to convert to string
    def __repr__(self):
        return str(self.dict)
    
    def __str__(self) -> str:
        return self.__repr__()
    
    def __eq__(self, other):
        return self.dict == other.dict
    
    def __hash__(self):
        return hash(
          str(list(sorted(self.dict.items())))
        )

d = Dict({"k": "v", "k2": "v2"})
d2 = Dict({"k2": "v2", "k": "v"})
set([d, d2])

{{'k': 'v', 'k2': 'v2'}}

In [21]:
set([1, 2, 4, False, "Hello", (1, 2, 3), 1.2322, 134343.12])

{(1, 2, 3), 1, 1.2322, 134343.12, 2, 4, False, 'Hello'}

In [24]:
class Entity:
  def __init__(self, name: str, time: int):
    self.name = name
    self.time = time
  
  def __repr__(self):
    return f'{self.name} ({self.time}s)'
  
  def __str__(self):
    return self.__repr__()
  
  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return self.name == other.name
  
  def __hash__(self):
    return hash(self.name)

a1 = Entity("Popeye", 12)
a2 = Entity("Popeye", 12)

a1 == a2
set([a1, a2])

{Popeye (12s)}

### L3

Help a family of 5 people move across the river by boat, and the boat is capable of a maximum of 2 people carrying capacity. Time for travelling of each person in turn is 1s, 3s, 6s, 8s, and 12s. If two people both go on the boat, the boat will travel at the speed of the slower person. Find the minimum time required for the whole family to cross the river. (less than eual to 30s).

[Sample gamplay video](https://www.youtube.com/watch?v=HdcQjI2vSao&list=PLXO4k3Jc8d7d892be-P_H9J_vE_TN73qa&index=2)

In [34]:
"""
Help a family of 5 people move across the river by boat, and the boat is capable of a maximum of 2 people carrying capacity. Time for travelling of each person in turn is 1s, 3s, 6s, 8s, and 12s. If two people both go on the boat, the boat will travel at the speed of the slower person. Find the minimum time required for the whole family to cross the river. (less than eual to 30s).
"""

from typing import List, Set
from functools import total_ordering

@total_ordering
class Entity:
  def __init__(self, name: str, time: int):
    self.name = name
    self.time = time
  
  def __repr__(self):
    return f'{self.name} ({self.time}s)'
  
  def __str__(self):
    return self.__repr__()
  
  # This method is required for comparison
  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return self.name == other.name
  
  # This method is required for hashing, and adding to set
  def __hash__(self):
    return hash(self.name)
  
  # This method is required for sorting
  def __lt__(self, other):
    return self.name < other.name


class State:
  def __init__(self, left: Set[Entity], right: Set[Entity], timeLeft: int, nextLeft2right: bool):
    self.left = left
    self.right = right
    self.timeLeft = timeLeft
    self.nextLeft2right = nextLeft2right
  
  def __repr__(self):
    return f'time: {self.timeLeft}, left: ({self.left}), right: ({self.right})'
  
  def __str__(self):
    return self.__repr__()
  
  def is_final(self):
    return len(self.left) == 0
  
  def is_valid(self):
    if self.timeLeft < 0:
      return False
    if self.left & self.right:
      return False
    if len(self.left) + len(self.right) != 5:
      return False
    return True
  
  # We need to make this class hashable 
  # to make sure we can insert a state into visited set.
  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return (
      self.left == other.left and
      self.right == other.right and
      self.timeLeft == other.timeLeft and
      self.nextLeft2right == other.nextLeft2right
    )
  
  def __hash__(self):
    return hash((
      str(sorted(self.left)),
      str(sorted(self.right)),
      self.timeLeft,
      self.nextLeft2right
    ))


class Move:
  def __init__(self, isLeft2right: bool, entities: Set[Entity]):
    if not 0 < len(entities) <= 2:
      raise ValueError(f'Invalid number of people crossing the river: Number of people should be 1 or 2, but got {len(entities)}')
    self.isLeft2right = isLeft2right
    self.source = 'left' if isLeft2right else 'right'
    self.destination = 'right' if isLeft2right else 'left'
    self.entities = entities
  
  def __repr__(self):
    return f'{"left" if self.isLeft2right else "right"} -> {"right" if self.isLeft2right else "left"}: {self.entities}'
  
  def __str__(self):
    return self.__repr__()
  
  def validate(self, state: State):
    source = state.left if self.isLeft2right else state.right
    destination = state.right if self.isLeft2right else state.left
    entities_not_in_source = self.entities - source
    entities_already_in_destination = self.entities & destination
    if self.isLeft2right != state.nextLeft2right:
      raise ValueError(f"""
        Invalid move: Expected {
        "left to right" if state.nextLeft2right else "right to left"
        } as next move, but got {
          "left to right" if self.isLeft2right else "right to left"}
        """)
    if entities_not_in_source:
      raise ValueError(f'Entities {entities_not_in_source} not in {self.source}. Current state: {state}')
    if entities_already_in_destination:
      raise ValueError(f'Entities {entities_already_in_destination} already in {self.destination}. Current state: {state}')
  
  def apply(self, state: State) -> State:
    # check if move is valid
    self.validate(state)
    # apply move
    left = state.left.copy()
    right = state.right.copy()
    if self.isLeft2right:
      left -= self.entities
      right |= self.entities
    else:
      left |= self.entities
      right -= self.entities
    timeLeft = state.timeLeft
    timeLeft -= max([entity.time for entity in self.entities])
    return State(left, right, timeLeft, not state.nextLeft2right)


class RiverCrossing:
  def __init__(self):
    self.entities = {
      Entity("Swee'Pea", 1),
      Entity("Popeye", 3),
      Entity("Olive", 6),
      Entity("Wimpy", 8),
      Entity("Wotasnozzle", 12)
    }
    self.initial_state = State(self.entities, set(), 30, True)
    self.visited = set()

  def get_possible_moves(self, state: State):
    for entity in state.left:
      yield Move(True, {entity})
      for entity1 in state.left:
        if entity == entity1:
          continue
        yield Move(True, {entity, entity1})
    for entity in state.right:
      yield Move(False, {entity})
      for entity1 in state.right:
        if entity == entity1:
          continue
        yield Move(False, {entity, entity1})

  def solve(self, state: State, moves: List[Move]):
    # print(state, moves)
    self.visited.add(state)
    if state.is_final():
      return moves
    for move in self.get_possible_moves(state):
      try:
        new_state = move.apply(state)
      except ValueError:
        continue
      if new_state and new_state.is_valid():
        if new_state in self.visited:
          continue
        result = self.solve(new_state, moves + [move])
        if result:
          return result
    return None

  def run(self):
    moves = self.solve(self.initial_state, [])
    if moves:
      print("Solution found:")
      for move in moves:
        print(move)
    else:
      print('No solution found')

RiverCrossing().run()

Solution found:
left -> right: {Swee'Pea (1s), Popeye (3s)}
right -> left: {Swee'Pea (1s)}
left -> right: {Swee'Pea (1s), Olive (6s)}
right -> left: {Swee'Pea (1s)}
left -> right: {Wimpy (8s), Wotasnozzle (12s)}
right -> left: {Popeye (3s)}
left -> right: {Swee'Pea (1s), Popeye (3s)}
